## Imports

In [1]:
import re
import numpy as np
import tensorflow as tf
import keras
import pandas as pd
from tensorflow.python.keras.activations import sigmoid

# utils.py

Contains relevant functions to rest of code. Vectorized, preprocessing, etc.

In [2]:
# scale between -15 and 15
def preprocess_scores(scores):
    for i, score in enumerate(scores):
        score = str(score).encode('utf-8')
        score = score.decode('utf-8-sig')
        
        if score[0:2] == '#+':
            score = 20
        elif score[0:2] == '#-':
            score = -20
        elif int(score) > 15:
            score = 15
        elif int(score) < -15:
            score = -15
        
        scores[i] = int(score)
    
    scores = scores.astype('float32')
    
    return scores

# return bit arrays from FEN string
def vectorize(fen):
    data = re.split(" ", fen)
    rows= re.split("/", data[0])
    turn = data[1]
    can_castle = data[2]
    passant = data[3]
    half_moves = data[4]
    full_moves = data[5]
    
    bit_vector = np.zeros((13, 8, 8), dtype=np.float32)
    
    #what layer each piece is found on
    piece_to_layer = {
            'R': 1,
            'N': 2,
            'B': 3,
            'Q': 4,
            'K': 5,
            'P': 6,
            'p': 7,
            'k': 8,
            'q': 9,
            'b': 10,
            'n': 11,
            'r': 12
        }
    #find each piece based on type
    for r,value in enumerate(rows):
        colum = 0
        for piece in value:
            if piece in piece_to_layer:
                bit_vector[piece_to_layer[piece],r,colum] =1
                colum += 1
            else:
                colum += int(piece)
    
    if turn.lower() == 'w':
        bit_vector [0,7,4] =1
    else:
        bit_vector [0,0,4] =1
        
    #where each castle bit is located
    castle ={
        'k': (0,0),
        'q': (0,7),
        'K': (7,0),
        'Q': (7,7),
        }

    for value in can_castle:
        if value in castle:
            bit_vector[0,castle[value][0],castle[value][1]] = 1
    
    #put en-passant square in the vector
    if passant != '-':
        bit_vector[0,  5 if (int(passant[1])-1 == 3) else 2 , ord(passant[0]) - 97,] = 1
    
    return bit_vector # we did a .reshape(something)

# sequenceTest.py

Contains code from relevant file

In [3]:
# Splitting data into banches so we can actually process this thing.
class DataSequence(keras.utils.Sequence):

    def __init__(self, file, batch_size):
        self.file = file
        self.batch_size = batch_size
      
    def __len__(self):
        #total_length = sum(1 for row in open(self.file))
        total_length = 100_000
        return int(np.ceil(total_length) / self.batch_size)
    
    
    def __getitem__(self, idx):
        df = pd.read_csv(self.file, skiprows=idx*self.batch_size, nrows=self.batch_size)
      
        x = np.ndarray(shape=(self.batch_size,13,8,8))
        y = np.array(df.iloc[:,1])
      
        #have preprocessing here right now for testing, should save processed scores in file
        # once we decide how to do this
        for i, f in enumerate(df.iloc[:,0]):
            x[i] = vectorize(f)
        x = x.reshape((self.batch_size,832))
        y = preprocess_scores(y)
        #print(x.shape, y.shape)
        #print(idx)
        
        return (x, y)
    
# does ?
def scaled_sigmoid(x):
    #print(2  * sigmoid(x) - 1)
    return 2  * sigmoid(x) - 1

# training.py

Contains code from relevant file

In [4]:
# Define the neural network architecture
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(13, 8, 8)),
    tf.keras.layers.Dense(832, activation='linear'),
    tf.keras.layers.Dense(832, activation='relu'),
    tf.keras.layers.Dense(832, activation='linear'),
    tf.keras.layers.Dense(832, activation='relu'),
    tf.keras.layers.Dense(832, activation='linear'),
    tf.keras.layers.Dense(832, activation='relu'),
    #tf.keras.layers.Dense(1, activation=scaled_sigmoid)
    tf.keras.layers.Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer='adam',
              loss='MeanAbsoluteError')

# df = pd.read_csv("../data/kaggleDataset/chessData.csv", 
#      nrows=10000, dtype={'FEN':str, 'Evaluation':str})

train_sequence = DataSequence("../data/kaggleDataset/chessData.csv", 1024)

### Trying to used 'vectorized.csv'

It is not returning in exactly the way we'd like, so I'm trying to deal with this here. Idk if this will work, that's why I need to try and plug it in to the neural network

In [5]:
# Converts the 'vectorized' things to the array type again. This feels like 2 steps backwards 1 forward
import numpy as np
import re

def toArray(fen_string):
    formatted_string = re.sub(r'\[|\]', '', fen_string)

    # Split the string into separate blocks of 13 8x8 arrays
    fen_blocks = formatted_string.split('\n\n ')

    # Define a function to convert a single FEN block to a 3D list
    def fen_to_3d_array(fen_block):
        rows = fen_block.split('\n')
        return [[[int(cell) for cell in row.split()] for row in rows[i:i + 8]] for i in range(0, len(rows), 8)]

    # Convert each FEN block to a 3D list
    fen_arrays = [fen_to_3d_array(fen_block) for fen_block in fen_blocks]

    # Convert to numpy array and check the shape
    return np.asarray(fen_arrays).reshape(13,8,8)

In [6]:
import pandas as pd
nrows = 50_000

df = pd.read_csv("../data/createdData/vectorized.csv", nrows=nrows)

In [7]:
# x
dfcopy = df.copy().head(100)[['FEN','Evaluation']]
x = dfcopy.FEN.apply(toArray)

# y
y = preprocess_scores(dfcopy.Evaluation)

In [8]:
np.asarray(x).reshape(100,13,8,8)

ValueError: cannot reshape array of size 100 into shape (100,13,8,8)

In [20]:
model.fit(dfcopy.FEN, dfcopy.Evaluation, epochs=10)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

Back to fitting the model

In [6]:
# Train the model
model.fit(train_sequence, epochs=10)

model.save('saved_model')

Epoch 1/10
97/97 [==============================] - 10s 90ms/step - loss: 11.7550
Epoch 2/10
97/97 [==============================] - 9s 92ms/step - loss: 10.1932
Epoch 3/10
97/97 [==============================] - 9s 91ms/step - loss: 9.4187
Epoch 4/10
97/97 [==============================] - 9s 97ms/step - loss: 8.8498
Epoch 5/10
97/97 [==============================] - 10s 103ms/step - loss: 7.9005
Epoch 6/10
97/97 [==============================] - 10s 104ms/step - loss: 7.1467
Epoch 7/10
97/97 [==============================] - 11s 109ms/step - loss: 6.2788
Epoch 8/10
97/97 [==============================] - 12s 119ms/step - loss: 5.5755
Epoch 9/10
97/97 [==============================] - 17s 177ms/step - loss: 4.9788
Epoch 10/10
97/97 [==============================] - 15s 150ms/step - loss: 4.4582
INFO:tensorflow:Assets written to: saved_model\assets


# runSavedBot.py

(Currently not evaluating)

Contains code from relevant file

In [7]:
# import tensorflow as tf
# import chess

# import utils

# def evaluate(board):
#     #print(vectorize(board.fen()).shape)
#     evaluation = model.predict_step(utils.vectorize(board.fen()).reshape((1,832)))
#     #print(evaluation)
#     return evaluation

# def find_best_move(board, depth, maximizing_player):
#     if maximizing_player:
#         best_score = float('-inf')
#         best_move = None
#         for move in board.legal_moves:
#             board.push(move)
#             score = minimax(board, depth - 1, float('-inf'), float('inf'), False)
#             print(move, score)
#             if score > best_score:
#                 best_score = score
#                 best_move = move
#             board.pop()
#     else:
#         best_score = float('inf')
#         best_move = None
#         for move in board.legal_moves:
#             board.push(move)
#             score = minimax(board, depth - 1, float('-inf'), float('inf'), True)
#             if score < best_score:
#                 best_score = score
#                 best_move = move
#             board.pop()
    
#     return best_move
    
# def minimax(board, depth, alpha, beta, maximizing_player):
#     if board.is_game_over() or depth == 0:
#         return evaluate(board)
    
#     if maximizing_player:
#         best_score = float('-inf')
#         for move in board.legal_moves:
#             board.push(move)
#             score = minimax(board, depth-1, alpha, beta, False)
#             #score += depth
#             board.pop()
#             best_score = max(score, best_score)
#             alpha = max(alpha, best_score)
#             if alpha >= beta:
#                 break
#         return best_score
#     else:
#         best_score = float('inf')
#         for move in board.legal_moves:
#             board.push(move)
#             score = minimax(board, depth-1, alpha, beta, True)
#             #score -= depth
#             board.pop()
#             best_score = min(score, best_score)
#             beta = min(beta, best_score)
#             if alpha >= beta:
#                 break
#         return best_score


# if __name__ == '__main__':
#     model = tf.keras.models.load_model('saved_model')
    
#     board = chess.Board()
#     depth = 3

#     while not board.is_game_over():
#         print(board)
#         print()

#         if board.turn == chess.WHITE:
#             move = find_best_move(board, depth, True)
#             print(move)
#         else:
#             move = chess.Move.from_uci(input(prompt='Move: '))
            

#         board.push(move)

#     print("Game over")
#     print("Result:", board.result())